# 16/05/2024
Notebook for listening to the attacks etc.

In [30]:
import IPython.display as ipd
import numpy as np

from src.audio_utils import read_audio
from src.utils import *
from sp_utils import spectrogram_inversion, get_spectrogram_from_audio
from attacks_utils import retrieve_single_cached_spec, load_spec_model, evaluate_spec, get_class

In [31]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [32]:
config_path = '../config/residualnet_train_config.yaml'
config = read_yaml(config_path)

rate = 16000

***
## Testing the audio pipeline
### Original phase available

In [33]:
index = 0
file, label, spec = retrieve_single_cached_spec(config, index)
audio, phase = spectrogram_inversion(config, index, spec, phase_info=True, phase_to_use=None)
ipd.Audio(audio, rate=rate)

Evaluating file /nas/public/dataset/asvspoof2021/ASVspoof2021_DF_eval/flac/DF_E_2000011.flac with label 1


### Original phase not available
SPSI + Griffin_Lim

In [34]:
file, label, spec = retrieve_single_cached_spec(config, index)
audio, phase = spectrogram_inversion(config, index, spec, phase_info=False, phase_to_use=None)
ipd.Audio(audio, rate=rate)

Evaluating file /nas/public/dataset/asvspoof2021/ASVspoof2021_DF_eval/flac/DF_E_2000011.flac with label 1


*** 
## Check the audios
Function for getting whatever type of audio file (evaluation or perturbed + the class label and the model prediction on the perturbed ones)

In [35]:
def get_audio_file(config, index, attack=False, epsilon=None):
    df_eval = pd.read_csv(os.path.join('..', config["df_eval_path"]))
    file_eval = list(df_eval['path'])
    label_eval = list(df_eval['label'])
    file = file_eval[index]
    label = label_eval[index]
    
    model = load_spec_model(device=device, config=config)
    model.eval()

    if attack:
        split_file = os.path.splitext(os.path.basename(file))[0]
        epsilon_str = str(epsilon).replace('.', 'dot')

        if attack == 'FGSM':
            folder = 'FGSM_data/p_audio'
        elif attack == 'FGSM_1':
            folder = 'FGSM_data/p_audio_1'

        p_file = os.path.join(folder, f'{attack}_{split_file}_{epsilon_str}.flac')
        spec = get_spectrogram_from_audio(p_file)
        out = evaluate_spec(spec, model, device)
        pred_class = get_class(out)

        print(f'Got the perturbed audio {p_file}. GT label is {label}. Model prediction is {pred_class}')
        X, sr = read_audio(p_file)

    else:
        spec = get_spectrogram_from_audio(file)
        out = evaluate_spec(spec, model, device)
        pred_class = get_class(out)
        print(f'Got the original audio {file} with GT label {label}. Model prediction is {pred_class}')
        X, sr = read_audio(file)

    return X, sr

In [36]:
index = 0
epsilon = 0.5

In [37]:
X, sr =get_audio_file(config, index=index)

Got the original audio /nas/public/dataset/asvspoof2021/ASVspoof2021_DF_eval/flac/DF_E_2000011.flac with GT label 1. Model prediction is 1


In [38]:
ipd.Audio(X, rate=sr)

In [39]:
X_, sr_ =get_audio_file(config, index=index, attack='FGSM_1', epsilon=epsilon)

Got the perturbed audio FGSM_data/p_audio_1/FGSM_1_DF_E_2000011_0dot5.flac. GT label is 1. Model prediction is 1


In [40]:
ipd.Audio(X_, rate=16000)

***
## Check the spectrograms

In [41]:
def get_spectrograms(config, index, attack=False, epsilon=None):
    model = load_spec_model(device=device, config=config)
    model.eval()
    
    if attack:
        df_eval = pd.read_csv(os.path.join('..', config["df_eval_path"]))
        file_eval = list(df_eval['path'])
        label_eval = list(df_eval['label'])
        file = file_eval[index]
        label = label_eval[index]
        
        split_file = os.path.splitext(os.path.basename(file))[0]
        epsilon_str = str(epsilon).replace('.', 'dot')
        
        if attack == 'FGSM':
            folder = 'FGSM_data/p_specs'
        elif attack == 'FGSM_1':
            folder = 'FGSM_data/p_specs_1'

        p_spec_path = os.path.join(folder, f'{attack}_{split_file}_{epsilon_str}.npy')
        p_spec = np.load(p_spec_path)
        out = evaluate_spec(p_spec, model, device)
        pred_class = get_class(out)
        print(f'Got the perturbed spectrogram {p_spec_path}. GT label is {label}. Model prediction is {pred_class}')
    
    else:
        file, label, spec = retrieve_single_cached_spec(config, index)
        out = evaluate_spec(spec, model, device)
        pred_class = get_class(out)
        print(f'Got the original spectrogram {file}. GT label is {label}. Model prediction is {pred_class}')
        

In [42]:
get_spectrograms(config, index, attack=False, epsilon=None)

Evaluating file /nas/public/dataset/asvspoof2021/ASVspoof2021_DF_eval/flac/DF_E_2000011.flac with label 1
Got the original spectrogram /nas/public/dataset/asvspoof2021/ASVspoof2021_DF_eval/flac/DF_E_2000011.flac. GT label is 1. Model prediction is 1


In [44]:
get_spectrograms(config, index, attack='FGSM_1', epsilon=epsilon)

Got the perturbed spectrogram FGSM_data/p_specs_1/FGSM_1_DF_E_2000011_0dot5.npy. GT label is 1. Model prediction is 0
